In [3]:
!git clone https://github.com/adhiiisetiawan/nbr-time_aware_item_weighting

fatal: destination path 'nbr-time_aware_item_weighting' already exists and is not an empty directory.


In [1]:
%cd nbr-time_aware_item_weighting/

e:\UI\0\_kalbe\nbr-time_aware_item_weighting


In [5]:
# !pip install -r requirements.txt

In [1]:
import os
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import BPR, SLRC, NBRKNN, RepurchaseModule
import torch
import random
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")

/home/adhi/nbr-time_aware_item_weighting/.env-nbr/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

In [5]:
corpus_path = "notebooks/data/"
dataset_name = "ta_feng"

corpus_path = "data/"
dataset_name = "EPM-prep"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(0, 0, filt=False)
# preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 465, #items = 1477, #clicks = 518655 (#illegal records = 0)
After preprocessing: #users = 465, #items = 1477, #clicks = 518655
Saving dataset in ..//data_EPM-prep/...


In [5]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

In [21]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=3,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


  0%|          | 0/465 [00:00<?, ?it/s]

100%|██████████| 465/465 [00:00<00:00, 1529.63it/s]


dev dataset preparing...


100%|██████████| 453/453 [00:06<00:00, 67.27it/s] 


test dataset preparing...


100%|██████████| 453/453 [00:06<00:00, 74.18it/s] 


In [22]:
slrc_best_params = {'batch_size': 256, 'lr': 0.00011201144001505824, 'l2_reg_coef': 0.00011498224071460201}

params = {
    "model": RepurchaseModule(
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)

In [23]:
trainer.train()

Epoch 1:


Batch loss = 0.488238: 100%|██████████| 2009/2009 [01:28<00:00, 22.68it/s]


Evaluation (dev):



100%|██████████| 453/453 [00:10<00:00, 44.67it/s]


 {'precision': 0.11655629139072848, 'recall': 0.3372051101577959, 'ndcg': 0.2539155788957893}
Epoch 2:


...


Batch loss = 0.482116: 100%|██████████| 2009/2009 [01:34<00:00, 21.37it/s]


Evaluation (dev):



100%|██████████| 453/453 [00:10<00:00, 41.99it/s]


 {'precision': 0.11677704194260487, 'recall': 0.3365376025366459, 'ndcg': 0.25207541161026886}


RepurchaseModule()

In [24]:
dev_user_emb = trainer.get_predictions(mode="dev")

100%|██████████| 453/453 [00:08<00:00, 51.72it/s] 


In [25]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 465/465 [00:00<00:00, 1226.94it/s]


dev dataset preparing...


100%|██████████| 453/453 [00:04<00:00, 100.86it/s]


test dataset preparing...


100%|██████████| 453/453 [00:04<00:00, 98.74it/s] 


In [26]:
def objective(trial):
    params = {
        "model": NBRKNN(
            item_num=corpus.n_items,
            user_num=corpus.n_users,
            nearest_neighbors_num=trial.suggest_int("nearest_neighbors_num", low=1, high=200),
            alpha=trial.suggest_float("alpha", 0.0, 1.0, step=0.05),
            user_emb=dev_user_emb
        )
    }

    trainer.init_hyperparams(**params)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=5)

[I 2023-10-31 09:08:51,019] A new study created in memory with name: no-name-69a01b52-a0f4-4cfe-b918-431318a56388
  0%|          | 0/453 [00:00<?, ?it/s]

100%|██████████| 453/453 [00:03<00:00, 137.46it/s]
[I 2023-10-31 09:08:54,454] Trial 0 finished with value: 0.23770489391879496 and parameters: {'nearest_neighbors_num': 110, 'alpha': 0.75}. Best is trial 0 with value: 0.23770489391879496.
100%|██████████| 453/453 [00:04<00:00, 111.63it/s]
[I 2023-10-31 09:08:58,613] Trial 1 finished with value: 0.21050125387768526 and parameters: {'nearest_neighbors_num': 121, 'alpha': 0.55}. Best is trial 0 with value: 0.23770489391879496.
100%|██████████| 453/453 [00:03<00:00, 122.25it/s]
[I 2023-10-31 09:09:02,399] Trial 2 finished with value: 0.22335214421666635 and parameters: {'nearest_neighbors_num': 85, 'alpha': 0.65}. Best is trial 0 with value: 0.23770489391879496.
100%|██████████| 453/453 [00:02<00:00, 158.84it/s]
[I 2023-10-31 09:09:05,330] Trial 3 finished with value: 0.25083966982148903 and parameters: {'nearest_neighbors_num': 88, 'alpha': 0.9}. Best is trial 3 with value: 0.25083966982148903.
100%|██████████| 453/453 [00:05<00:00, 87.1

In [27]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [28]:
seed = 0
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=3,
    topk=10,
    early_stop_num=3
)

params = {
    "model": RepurchaseModule(
        item_num=corpus.n_items,
        avg_repeat_interval=corpus.total_avg_interval
    ),
    "batch_size": slrc_best_params["batch_size"],
    "lr": slrc_best_params["lr"],
    "l2_reg_coef": slrc_best_params["l2_reg_coef"]
}

trainer.init_hyperparams(**params)
trainer.train()

dev_user_emb = trainer.get_predictions(mode="dev")
test_user_emb = trainer.get_predictions(mode="test")

trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=None,
    topk=10,
    early_stop_num=None
)

params = {
    "model": NBRKNN(
        item_num=corpus.n_items,
        user_num=corpus.n_users,
        nearest_neighbors_num=study.best_params["nearest_neighbors_num"],
        alpha=study.best_params["alpha"],
        user_emb=dev_user_emb
    )
}
params["model"].set_emb(test_user_emb)

trainer.init_hyperparams(**params)

metrics = trainer.evaluate(mode="test")

test_metrics["precision"].append(metrics["precision"])
test_metrics["recall"].append(metrics["recall"])
test_metrics["ndcg"].append(metrics["ndcg"])

train dataset preparing...


100%|██████████| 465/465 [00:00<00:00, 1462.28it/s]


dev dataset preparing...


100%|██████████| 453/453 [00:06<00:00, 70.84it/s] 


test dataset preparing...


...

100%|██████████| 465/465 [00:00<00:00, 2313.49it/s]


dev dataset preparing...


100%|██████████| 453/453 [00:04<00:00, 91.20it/s] 


test dataset preparing...


100%|██████████| 453/453 [00:02<00:00, 157.89it/s]


In [29]:
test_metrics

{'precision': [0.11479028697571744],
 'recall': [0.36210864303198126],
 'ndcg': [0.2735911520620894]}

In [3]:
import pandas as pd

data = pd.read_csv('data/EPM.csv')

In [67]:
def inference(user, topk):
    n_items = corpus.n_items

    items_scores = trainer.model.predict_for_user(user).view(-1, n_items)
    top_items = torch.topk(items_scores, k=topk, dim=1, sorted=True).indices
    top_score = [float(i) for i in items_scores[0]]

    print("Item  | Score")
    print('-------------')
    for i in top_items[0]:
        print("{:5} | {:<5}".format(sorted(data['item_code'].unique())[int(i)], round(top_score[i], 3)))

In [73]:
# check id of customer
sorted(data['customer_name'].unique())[114]

'JK1-AP. DEVITA_GROUP_NA'

In [74]:
customer_id = 114
top_k = 5

inference(customer_id, top_k)

Item  | Score
-------------
DVCOA | 4.115
CPL50 | 2.747
CKCOA | 2.215
CRTDS | 2.005
10224 | 1.977


'10223'